# Start work with data

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
data = pd.read_csv("diabetes.csv")
data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


# Gini Impurity

In [2]:
class Node: 
    #Constructor for the node, used for root node an recursive calls
    def __init__(
        self, 
        Y: pd.DataFrame,
        X: pd.DataFrame,
        min_samples_split=None,
        max_depth=None,
        curr_depth=None):
        # Saving the data to the node 
        self.Y = Y
        self.X = X
        #Default min split to 20
        self.min_samples_split = min_samples_split if min_samples_split else 20
        #Default max depth to 5
        self.max_depth = max_depth if max_depth else 5
        self.curr_depth = curr_depth if curr_depth else 0
        self.features = list(self.X.columns)
        self.counts = Counter(Y)
        self.gini_impurity = self.get_GINI()
        counts_sorted = list(sorted(self.counts.items(), key=lambda item: item[1]))
        preciction_value = None
        if len(counts_sorted) > 0:
            preciction_value = counts_sorted[-1][0]
        self.preciction_value = preciction_value
        self.n = len(Y)
        
        # Initiating the left and right nodes as empty nodes
        self.left = None 
        self.right = None 
        self.best_feature = None 
        self.best_value = None 

    ##Gets the Gini Impurity
    @staticmethod
    def get_impurity(y1_count: int, y2_count: int) -> float:
        y1_count = 0 if y1_count is None else y1_count
        y2_count = 0 if y2_count is None else y2_count
        # Total observations
        n = y1_count + y2_count
        # If n is 0 then its the lowest score possible
        if n == 0:
            return 0.0
        p1 = y1_count / n
        p2 = y2_count / n
        gini = 1 - (p1 ** 2 + p2 ** 2)
        return gini

    ##Calculate the moving average
    @staticmethod
    def moving_average(x: np.array, window: int) -> np.array:
        return np.convolve(x, np.ones(window), 'valid') / window

    ##Calculated the Gini impurity
    def get_GINI(self):
        y1_count, y2_count = self.counts.get(0, 0), self.counts.get(1, 0)
        return self.get_impurity(y1_count, y2_count)

    #Predicts the best split for the node
    def best_split(self) -> tuple:
        df = self.X.copy()
        df['Y'] = self.Y
        GINI_base = self.get_GINI()
        max_gain = 0
        best_feature = None
        best_value = None
        for feature in self.features:
            
            sorted_df = df.sort_values(feature)
            x_means = self.moving_average(sorted_df[feature].unique(), 2)
            for value in x_means:
                # Spliting the dataset 
                left_counts = Counter(sorted_df[sorted_df[feature]<value]['Y'])
                right_counts = Counter(sorted_df[sorted_df[feature]>=value]['Y'])
                y0_left=left_counts.get(0, 0)
                y1_left=left_counts.get(1, 0)
                y0_right=right_counts.get(0, 0)
                y1_right= right_counts.get(1, 0)
                gini_left = self.get_impurity(y0_left, y1_left)
                gini_right = self.get_impurity(y0_right, y1_right)
                
                # Getting the observation count from the left and the right data
                obs_left = y0_left + y1_left
                obs_right = y0_right + y1_right
                w_left = obs_left / (obs_left + obs_right)
                w_right = obs_right / (obs_left + obs_right)

                wGINI = w_left * gini_left + w_right * gini_right
                GINIgain = GINI_base - wGINI
                if GINIgain > max_gain:
                    best_feature = feature
                    best_value = value 
                    max_gain = GINIgain
        return (best_feature, best_value)

    ## Generated the tree from the root, is used recursivly
    def generate_tree(self):
        # Copy Data
        df = self.X.copy()
        df['Y'] = self.Y
        # Cutoff tests
        if (self.curr_depth < self.max_depth) and (self.n >= self.min_samples_split):
            best_feature, best_value = self.best_split()
            if best_feature is not None:
                self.best_feature = best_feature
                self.best_value = best_value
                left_df, right_df = df[df[best_feature]<=best_value].copy(), df[df[best_feature]>best_value].copy()
                left = Node(
                    left_df['Y'], 
                    left_df[self.features], 
                    curr_depth=self.curr_depth + 1, 
                    max_depth=self.max_depth, 
                    min_samples_split=self.min_samples_split, 
                    )
                self.left = left 
                self.left.generate_tree()
                right = Node(
                    right_df['Y'].values.tolist(), 
                    right_df[self.features], 
                    curr_depth=self.curr_depth + 1, 
                    max_depth=self.max_depth, 
                    min_samples_split=self.min_samples_split,
                    )
                self.right = right
                self.right.generate_tree()

    ## Predicts the node
    def predict(self, X:pd.DataFrame):
        predictions = []
        for _, x in X.iterrows():
            values = {}
            for feature in self.features:
                values.update({feature: x[feature]})
            ## Adding to predictions
            predictions.append(self.predict_obs(values))
        return predictions

    ##Predict given features
    def predict_obs(self, values: dict) -> int:
        cur_node = self
        while cur_node.curr_depth < cur_node.max_depth:
            best_feature = cur_node.best_feature
            best_value = cur_node.best_value
            if cur_node.n < cur_node.min_samples_split:
                break 
            if (values.get(best_feature) < best_value):
                if self.left is not None:
                    cur_node = cur_node.left
            else:
                if self.right is not None:
                    cur_node = cur_node.right
            
        return cur_node.preciction_value
        

# Test Data

In [14]:
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn import metrics
feature_cols = ['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age']
X = data[feature_cols] # Features
y = data['Outcome'] # Target variable
X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.25, random_state=0)

In [17]:
# Initiating the Root node
root = Node(y_train, X_train, max_depth=5, min_samples_split=100)

# Getting the best split
root.generate_tree()

# Predicting 
custom_predictions = root.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, custom_predictions))
print("Error:",(1-metrics.accuracy_score(y_test, custom_predictions)))

Accuracy: 0.7760416666666666
Error: 0.22395833333333337


In [18]:
##Sk learns tree for comparison
X = data[feature_cols] # Features
y = data['Outcome'] # Target variable
X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.25, random_state=0)
sklearn_tree = tree.DecisionTreeClassifier(max_depth=5,min_samples_split=100)
sklearn_tree.fit(X_train,y_train)
sklearn_predict = sklearn_tree.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, sklearn_predict))
print("Error:",(1-metrics.accuracy_score(y_test, sklearn_predict)))

Accuracy: 0.78125
Error: 0.21875
